In [2]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import zipfile, io
import pandas as pd
from time import strptime
from datetime import datetime
from sklearn import linear_model
from sklearn.linear_model import LinearRegression,Ridge,Lasso, RandomizedLasso
import requests
from requests import session
import numpy as np
import math
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [3]:
from sklearn import *
from sklearn.cross_validation import train_test_split
from sklearn.metrics import *
from IPython.display import HTML, display
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets, linear_model
from sklearn.neural_network import MLPRegressor
from sknn.mlp import Regressor, Layer

/gpfs/fs01/user/se3f-812b629ef574a0-aa895337ea30/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/gpfs/fs01/user/se3f-812b629ef574a0-aa895337ea30/.local/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/gpfs/fs01/user/se3f-812b629ef574a0-aa895337ea30/.local/lib/python3.5/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnal

In [66]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_37f4b508255f4f0187ff7d11d2ac7164(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_9f8ae4124ed5935b494820bc4caad226a8faaec9','domain': {'id': '3a7926077f894d6098a1599233ed725d'},
            'password': 'b5P&=(MSdQ?_ryx4'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_37f4b508255f4f0187ff7d11d2ac7164('Lendingclub', 'Lending_ModelFile.csv'))
df_data_1.head()

/usr/local/src/conda3_runtime/4.1.1/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_title,...,purpose_num,application_type_num,loan_status_num,issue_d_year,issue_d_month,earliest_cr_line_year,earliest_cr_line_month,last_credit_pull_month,last_credit_pull_year,last_credit_pull_monthYear
0,0,1077501,5000.0,5000.0,4975.0,36.0,10.65,162.87,B,Job Title not given,...,1,1,1.0,2011,12,1985,1,3,2017,3/2017
1,1,1077430,2500.0,2500.0,2500.0,60.0,15.27,59.83,C,Ryder,...,2,1,2.0,2011,12,1999,4,10,2016,10/2016
2,2,1077175,2400.0,2400.0,2400.0,36.0,15.96,84.33,C,Job Title not given,...,3,1,1.0,2011,12,2001,11,3,2017,3/2017
3,3,1076863,10000.0,10000.0,10000.0,36.0,13.49,339.31,C,AIR RESOURCES BOARD,...,4,1,1.0,2011,12,1996,2,4,2016,4/2016
4,4,1075358,3000.0,3000.0,3000.0,60.0,12.69,67.79,B,University Medical Group,...,4,1,1.0,2011,12,1996,1,1,2017,1/2017


In [67]:
#Drop extra columns
def dropExtraColumns(df):
    df = df.drop(['earliest_cr_line_month','last_credit_pull_month','mo_sin_rcnt_rev_tl_op','title','id',
                        'mths_since_recent_bc','num_accts_ever_120_pd','num_actv_bc_tl',
                        'num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts',
                        'num_rev_tl_bal_gt_0','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m',
                        'pct_tl_nvr_dlq','emp_title','mo_sin_rcnt_tl','id','percent_bc_gt_75',
                        'total_rec_late_fee','zip_code'],axis=1)
    return df

In [68]:
#Derive new columns
def deriveColumn(df):
    df['credit_history']= (df['last_credit_pull_year'] - df['earliest_cr_line_year'])
    df['grade_On_Inquiries']=np.where((df['inq_last_6mths']==0),'A', np.where(df['inq_last_6mths'].between(1,2), 'B',
           np.where(df['inq_last_6mths'].between(3,6), 'C',
           np.where(df['inq_last_6mths'].between(7,10), 'D','E'))))
    df = df[df.loan_status_num!= 8 ]
    df['UPB'] = (df['loan_amnt'] - df['total_rec_prncp']+df['total_rec_int'])
    df.loan_status_num.fillna(1,inplace=True)
    return df
    

In [69]:
df_data_1 = dropExtraColumns(df_data_1)

In [70]:
df_data_1 = deriveColumn(df_data_1)

In [9]:
df_data_1['grade_On_Inquiries'] = pd.factorize(df_data_1.grade_On_Inquiries)[0]

In [77]:
df_data_1.corr()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,...,application_type_num,loan_status_num,issue_d_year,issue_d_month,earliest_cr_line_year,earliest_cr_line_month,last_credit_pull_month,last_credit_pull_year,credit_history,grade_On_Inquiries
Unnamed: 0,1.000000,-0.241488,0.032021,0.032931,0.034571,0.040296,-0.007727,0.018651,0.007579,-0.007974,...,-0.056407,0.069333,-0.008537,-0.529875,-0.035369,-0.000419,0.017682,0.020538,0.037333,-0.000098
id,-0.241488,1.000000,0.021643,0.023237,0.026389,-0.005537,-0.074034,0.016515,-0.005260,0.039190,...,0.093668,-0.170017,0.896724,0.143871,0.117473,-0.002436,-0.284080,0.409835,-0.076670,0.073299
loan_amnt,0.032021,0.021643,1.000000,0.999542,0.998505,0.398112,0.142841,0.946518,0.120921,0.322301,...,0.028881,0.020064,0.056238,-0.024312,-0.168011,-0.013057,-0.011425,0.044712,0.172107,0.029519
funded_amnt,0.032931,0.023237,0.999542,1.000000,0.999057,0.397436,0.142929,0.947169,0.121022,0.322201,...,0.028978,0.019740,0.059479,-0.024695,-0.167552,-0.013079,-0.012260,0.046904,0.171866,0.029689
funded_amnt_inv,0.034571,0.026389,0.998505,0.999057,1.000000,0.397702,0.143069,0.946071,0.121554,0.321864,...,0.029085,0.018761,0.067714,-0.024274,-0.166670,-0.013143,-0.013625,0.053833,0.171671,0.030079
term,0.040296,-0.005537,0.398112,0.397436,0.397702,1.000000,0.411325,0.137639,0.076761,0.056713,...,0.015756,0.071688,0.015515,0.000783,-0.048280,-0.000768,-0.010847,0.044076,0.052549,0.000756
int_rate,-0.007727,-0.074034,0.142841,0.142929,0.143069,0.411325,1.000000,0.146934,-0.002988,-0.078792,...,0.059135,0.182120,-0.050487,0.016985,0.115196,0.001498,0.055450,-0.009715,-0.115931,-0.170485
installment,0.018651,0.016515,0.946518,0.947169,0.946071,0.137639,0.146934,1.000000,0.103409,0.311648,...,0.033039,0.023272,0.049123,-0.024000,-0.144585,-0.013152,-0.001297,0.032269,0.147495,0.007315
emp_length,0.007579,-0.005260,0.120921,0.121022,0.121554,0.076761,-0.002988,0.103409,1.000000,0.090984,...,-0.006247,-0.017831,0.012882,-0.003298,-0.133849,-0.014206,-0.014223,0.029587,0.136515,0.002425
annual_inc,-0.007974,0.039190,0.322301,0.322201,0.321864,0.056713,-0.078792,0.311648,0.090984,1.000000,...,-0.025012,-0.035746,0.040852,0.001731,-0.136518,-0.015670,-0.021274,0.020688,0.138298,-0.013481


In [10]:
df_data_1.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,...,application_type_num,loan_status_num,issue_d_year,issue_d_month,earliest_cr_line_year,last_credit_pull_year,last_credit_pull_monthYear,credit_history,grade_On_Inquiries,UPB
0,0,5000.0,5000.0,4975.0,36.0,10.65,162.87,B,10,RENT,...,1,1.0,2011,12,1985,2017,3/2017,32,0,863.16
1,1,2500.0,2500.0,2500.0,60.0,15.27,59.83,C,0,RENT,...,1,2.0,2011,12,1999,2016,10/2016,17,1,2478.71
2,2,2400.0,2400.0,2400.0,36.0,15.96,84.33,C,10,RENT,...,1,1.0,2011,12,2001,2017,3/2017,16,0,605.67
3,3,10000.0,10000.0,10000.0,36.0,13.49,339.31,C,10,RENT,...,1,1.0,2011,12,1996,2016,4/2016,20,0,2214.92
4,4,3000.0,3000.0,3000.0,60.0,12.69,67.79,B,1,RENT,...,1,1.0,2011,12,1996,2017,1/2017,21,2,1066.91


In [159]:
#Model Metrics dataframe
model_metric = pd.DataFrame({'AcceptedLoan_Model':[],'RMS_train':[],'RMS_test': [],'MAE_train': [],'MAE_test':[],'RMSE_Test':[]})
                             #'MAPE_train':[],'MAPE_test':[]})
RMSE_dict = {}

In [160]:
def ModelMetrics(modelname, model, X_train, y_train, X_test, y_test):
    ytrain_pred = model.predict(X_train)
    ytest_pred = model.predict(X_test)
    RMS_train = mean_squared_error(y_train, ytrain_pred)
    RMS_test = mean_squared_error(y_test, ytest_pred)
    MAE_train = mean_absolute_error(y_train, ytrain_pred)
    MAE_test = mean_absolute_error(y_test, ytest_pred)
    RMSE_Test = np.sqrt(mean_squared_error(y_test, ytest_pred))
    #MAPE_train = np.mean(np.abs((y_train - ytrain_pred) / y_train)) * 100
    #MAPE_test = np.mean(np.abs((y_test - ytest_pred) / y_test)) * 100
    RMSE_dict[modelname] = RMS_test
    df_metrics = pd.DataFrame({'AcceptedLoan_Model':[modelname],'RMS_train':[RMS_train],'RMS_test': [RMS_test],'MAE_train': [MAE_train],'MAE_test':[MAE_test],'RMSE_Test':[RMSE_Test]})
                               #'MAPE_train':[MAPE_train],'MAPE_test':[MAPE_test]})   
    global model_metric
    model_metric = pd.concat([model_metric, df_metrics])
    print("Completed",modelname)
    return (model_metric,df_metrics)


In [161]:
##df_data_1['loan_status'] = pd.factorize(df_data_1.loan_status)[0]#
df_data_1['home_ownership'] = pd.factorize(df_data_1.home_ownership)[0]
df_data_1['verification_status'] = pd.factorize(df_data_1.verification_status)[0]
df_data_1['purpose'] = pd.factorize(df_data_1.purpose)[0]


In [162]:
#collecting columns which are candidates for predicting interest rate
#feature_cols = ['term','inq_last_6mths','application_type_num','delinq_2yrs','home_ownership_num', 'revol_util','ficoMean','loan_amnt',
#                'purpose_num','emp_length','annual_inc','open_acc','credit_history','dti','installment']
#acc_now_delinq+loan_amnt+inq_last_6mths+term+annual_inc+revol_util+ficoMean+installment++loan_status_num+credit_history+open_acc

#'term','ficoMean','loan_amnt','installment','open_acc','credit_history','revol_util','inq_last_6mths','funded_amnt','out_prncp' --RMSE:0.6
#'term','ficoMean','loan_amnt','installment','emp_length','annual_inc','dti','purpose','open_acc','credit_history','funded_amnt','revol_util','inq_last_6mths','out_prncp' -- RMSE: 1.48

feature_cols = ['term',
                'ficoMean',
                'loan_amnt',
                'installment',
                'home_ownership_num',
                'annual_inc',
                'emp_length',
                'purpose_num',
                'open_acc',
                'credit_history',
                'revol_util',
                'inq_last_6mths',
                 'addr_state',
                'loan_status_num']


In [163]:
#Split into training and testing datasets
X = df_data_1[feature_cols]
y = df_data_1.int_rate
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [103]:
#Feature Selection: RFE
df_X = df_data_1[feature_cols]
df_y = df_data_1[[5]]


In [104]:
lmReg = LinearRegression()
rfe = RFE(estimator=lmReg, n_features_to_select=12, step=1)

In [105]:
rfe.fit(df_X,df_y)

/gpfs/fs01/user/se3f-812b629ef574a0-aa895337ea30/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  n_features_to_select=12, step=1, verbose=0)

In [106]:
ranking = rfe.ranking_

In [108]:
print(sorted(zip(map(lambda x: round(x, 2), rfe.ranking_), df_X)))

[(1, 'credit_history'), (1, 'emp_length'), (1, 'ficoMean'), (1, 'home_ownership_num'), (1, 'inq_last_6mths'), (1, 'installment'), (1, 'loan_amnt'), (1, 'loan_status_num'), (1, 'open_acc'), (1, 'purpose_num'), (1, 'revol_util'), (1, 'term'), (2, 'addr_state'), (3, 'annual_inc')]


In [164]:
#Linear Regression
def linearRegression(X_train,y_train,X_test,y_test):
    print("Linear Regression model computation starts")
    lm_model = LinearRegression()
    lm_model.fit(X_train, y_train)
    print("Coefficient is:",lm_model.coef_)
    print("Intercept is:",lm_model.intercept_)
    train_pred = lm_model.predict(X_train)
    print("R-Square Training",r2_score(y_train,train_pred))      
    test_pred = lm_model.predict(X_test)
    print("R-Square Testing",r2_score(y_test, test_pred)) 
    ModelMetrics('Regression',lm_model, X_train, y_train, X_test, y_test)
    print(model_metric)
    return('Linear Regression',lm_model)

In [165]:
algo_name,model = linearRegression(X_train,y_train,X_test,y_test)

Linear Regression model computation starts
Coefficient is: [  4.62297842e-01  -3.16846012e-02  -1.36464427e-03   4.47008055e-02
  -2.09305350e-01  -3.79871032e-06  -1.09434443e-02   1.89022022e-01
  -1.94375285e-02  -3.18253477e-02   1.16685630e-02   6.31845010e-01
   2.28157726e-04   9.39023705e-01]
Intercept is: 14.5688203328
R-Square Training 0.614847567625
R-Square Testing 0.615626946082
Completed Regression
  AcceptedLoan_Model  MAE_test  MAE_train  RMSE_Test  RMS_test  RMS_train
0         Regression   2.20135   2.204143   2.834695  8.035495   8.057798


In [175]:
#Ordinary Least Square Assumption
ols_model = ols("int_rate ~df_data_1[feature_cols]", data=df_data_1).fit()


In [176]:
ols_model_summary = ols_model.summary()


In [177]:
HTML(
ols_model_summary\
.as_html()\
.replace(' Adj. R-squared: ', ' Adj. R-squared: ')\
.replace('coef', 'coef')\
.replace('std err', 'std err')\
.replace('P>|t|', 'P>|t|')\
.replace('[95.0% Conf. Int.]', '[95.0% Conf. Int.]')
)

In [166]:
##Random Forest Algorithm
def randomForest(X_train,y_train,X_test,y_test):
    print("Random Forest computation starts")
    rForest = RandomForestRegressor(n_estimators=20)
    rForest.fit(X_train, y_train)
    train_pred = rForest.predict(X_train)
    print("R-Square Training",r2_score(y_train,train_pred))      
    test_pred = rForest.predict(X_test)
    print("R-Square Testing",r2_score(y_test, test_pred)) 
    ModelMetrics('Random Forest',rForest, X_train, y_train, X_test, y_test)
    print(model_metric)
    return('Random Forest',rForest)

In [167]:
algo_name,model =  randomForest(X_train,y_train,X_test,y_test)

Random Forest computation starts
R-Square Training 0.988811861218
R-Square Testing 0.933244807966
Completed Random Forest
  AcceptedLoan_Model  MAE_test  MAE_train  RMSE_Test  RMS_test  RMS_train
0         Regression  2.201350   2.204143   2.834695  8.035495   8.057798
0      Random Forest  0.552383   0.222355   1.181333  1.395548   0.234068


In [168]:
print(pd.DataFrame({'feature':feature_cols, 'importance':model.feature_importances_}).sort('importance'))

/usr/local/src/conda3_runtime/4.1.1/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


               feature  importance
4   home_ownership_num    0.002330
13     loan_status_num    0.002354
6           emp_length    0.005210
12          addr_state    0.008528
8             open_acc    0.009193
9       credit_history    0.011899
10          revol_util    0.020101
11      inq_last_6mths    0.024274
7          purpose_num    0.028694
5           annual_inc    0.037087
1             ficoMean    0.154869
0                 term    0.169313
2            loan_amnt    0.222890
3          installment    0.303258


In [169]:
##KNN Algorithm
def knnRegression(X_train, y_train, X_test, y_test):
    print("Knn Regression computation starts")
    knn = KNeighborsRegressor(n_neighbors=3)
    knn.fit(X_train,y_train)     
    test_pred = knn.predict(X_test)
    print("R-Square Testing",r2_score(y_test, test_pred)) 
    ModelMetrics('KNN',knn, X_train, y_train, X_test, y_test)
    print(model_metric)
    return('KNN',knn)

In [170]:
algo_name, model = knnRegression(X_train, y_train, X_test, y_test)

Knn Regression computation starts
R-Square Testing 0.624362133807
Completed KNN
  AcceptedLoan_Model  MAE_test  MAE_train  RMSE_Test  RMS_test  RMS_train
0         Regression  2.201350   2.204143   2.834695  8.035495   8.057798
0      Random Forest  0.552383   0.222355   1.181333  1.395548   0.234068
0                KNN  1.672386   1.137260   2.802299  7.852882   3.667681


In [171]:
##Neural Network Algorithm

In [172]:
def NeuralNetwork(X_train, y_train,X_test,y_test):
    print("Neural Network computation starts")
    scaler = StandardScaler()
    # Fit training data
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    nn = MLPRegressor()
    nn.fit(X_train,y_train)     
    test_pred = nn.predict(X_test)
    print("R-Square Testing",r2_score(y_test, test_pred)) 
    ModelMetrics('Neural Network',nn, X_train, y_train, X_test, y_test)
    print(model_metric)
    return('Neural Network',nn)    

In [173]:
algo_name, model = NeuralNetwork(X_train, y_train,X_test,y_test)

Neural Network computation starts
R-Square Testing 0.994069601992
Completed Neural Network
  AcceptedLoan_Model  MAE_test  MAE_train  RMSE_Test  RMS_test  RMS_train
0         Regression  2.201350   2.204143   2.834695  8.035495   8.057798
0      Random Forest  0.552383   0.222355   1.181333  1.395548   0.234068
0                KNN  1.672386   1.137260   2.802299  7.852882   3.667681
0     Neural Network  0.160248   0.159514   0.352105  0.123978   0.122409


In [174]:
model_metric

,AcceptedLoan_Model,MAE_test,MAE_train,RMSE_Test,RMS_test,RMS_train
0,Regression,2.201350,2.204143,2.834695,8.035495,8.057798
0,Random Forest,0.552383,0.222355,1.181333,1.395548,0.234068
0,KNN,1.672386,1.137260,2.802299,7.852882,3.667681
0,Neural Network,0.160248,0.159514,0.352105,0.123978,0.122409


In [178]:
#Find Best model
def BestModel(df):
    df['Model_Rank'] = df['RMS_test'].rank(ascending=1)
    print("Model with Model_Rank as 1 is Best Model")
    return df


In [179]:
BestModel(model_metric)

Model with Model_Rank as 1 is Best Model


,AcceptedLoan_Model,MAE_test,MAE_train,RMSE_Test,RMS_test,RMS_train,Model_Rank
0,Regression,2.201350,2.204143,2.834695,8.035495,8.057798,4.0
0,Random Forest,0.552383,0.222355,1.181333,1.395548,0.234068,2.0
0,KNN,1.672386,1.137260,2.802299,7.852882,3.667681,3.0
0,Neural Network,0.160248,0.159514,0.352105,0.123978,0.122409,1.0


In [71]:
#Create clusters using term and home_ownership
df_data_1.home_ownership.replace('NONE','OTHER', inplace=True)
df_data_1.home_ownership.replace('ANY','OTHER', inplace=True)
df_test = {}
df_test2= {}

g1 = df_data_1.groupby('term')
list_keys = [ k for k in g1.groups.keys() ]

i=0
count = 0
while i<len(list_keys):
    df_test[i]=g1.get_group(list_keys[i]).reset_index()  
    g2 = df_test[i].groupby('home_ownership')
    list_keys2 = [ k for k in g2.groups.keys() ]
    j=0
    while j<len(list_keys2):
        df_test2[j]=g2.get_group(list_keys2[j]).reset_index()
        fileName = 'bin' + str(count) + '.csv'
        print(fileName)
        df_test2[j].to_csv(fileName,index=False)
        j=j+1
        count=count+1
    i=i+1

bin0.csv
bin1.csv
bin2.csv
bin3.csv
bin4.csv
bin5.csv
bin6.csv
bin7.csv


In [150]:
df_bin.home_ownership.replace('NONE','OTHER', inplace=True)
df_bin.home_ownership.replace('ANY','OTHER', inplace=True)
df_bintest = {}
df_bintest2= {}

g1 = df_bin.groupby('term')
list_keys = [ k for k in g1.groups.keys() ]

i=0
count = 0
while i<len(list_keys):
    df_bintest[i]=g1.get_group(list_keys[i]).reset_index()  
    g2 = df_bintest[i].groupby('home_ownership')
    list_keys2 = [ k for k in g2.groups.keys() ]
    j=0
    while j<len(list_keys2):
        df_bintest2[j]=g2.get_group(list_keys2[j]).reset_index()
        fileName = 'newbin' + str(count) + '.csv'
        print(fileName)
        df_bintest2[j].to_csv(fileName,index=False)
        j=j+1
        count=count+1
    i=i+1

newbin0.csv
newbin1.csv
newbin2.csv
newbin3.csv
newbin4.csv
newbin5.csv
newbin6.csv
newbin7.csv


In [72]:
# @hidden_cell
credentials_38 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_37f4b508_255f_4f01_87ff_7d11d2ac7164',
  'project_id':'5f7d1f48e2d94194b0cddd68902dd993',
  'region':'dallas',
  'user_id':'b0aee163cb904664afb3a41ec599075c',
  'domain_id':'3a7926077f894d6098a1599233ed725d',
  'domain_name':'1256591',
  'username':'member_9f8ae4124ed5935b494820bc4caad226a8faaec9',
  'password':"""b5P&=(MSdQ?_ryx4""",
  'container':'Lendingclub',
  'tenantId':'undefined',
  'filename':'Model_Term60.csv'
}

In [73]:
#For saving CSV Files
from io import BytesIO  
import requests  
import json  
import pandas as pd

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print(resp2)

In [158]:
put_file(credentials_38, 'newbin7.csv')

<Response [201]>


In [ ]:
#drop extra columns
df_data_1 = df_data_1.drop(['funded_amnt_inv','last_credit_pull_month','mo_sin_rcnt_rev_tl_op','title','id',
                        'mths_since_recent_bc','num_accts_ever_120_pd','num_actv_bc_tl',
                        'num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts',
                        'num_rev_tl_bal_gt_0','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m',
                        'pct_tl_nvr_dlq','emp_title','mo_sin_rcnt_tl','id','percent_bc_gt_75',
                        'total_rec_late_fee','zip_code'],axis=1)

In [127]:
df_bin = pd.DataFrame(df_data_1)

In [146]:
df_bin = df_bin.drop(['total_rec_prncp',
 'total_rec_int','total_rev_hi_lim',
 'tot_hi_cred_lim','UPB'],axis=1)

In [147]:
list(df_bin)

['loan_amnt',
 'funded_amnt',
 'term',
 'int_rate',
 'installment',
 'grade',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'issue_d',
 'loan_status',
 'purpose',
 'addr_state',
 'dti',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'open_acc',
 'revol_util',
 'out_prncp',
 'last_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'ficoMean',
 'last_ficoMean',
 'grade_num',
 'home_ownership_num',
 'purpose_num',
 'loan_status_num',
 'issue_d_year',
 'earliest_cr_line_year',
 'last_credit_pull_year',
 'last_credit_pull_monthYear',
 'credit_history',
 'grade_On_Inquiries']

In [148]:
df_bin.shape

(1319098, 40)